In [47]:
import numpy as np
import networkx as nx
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import Tsp
import csv

locations = []

with open('oceanPollutionData.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        location = list(row.values())[0]
        if location not in locations:
            locations.append(location)
print(f"Unique locations found: {locations}")

n_cities = len(locations)
graph = nx.complete_graph(n_cities)
adj_matrix = np.array(
    [[0, 36.2048, 138.2529], #north east 
     [39.0742, 0, 21.8243],
     [55.3781, 3.4360, 0]]
)

for i in range(n_cities):
    for j in range(n_cities):
        if i != j:
            graph.edges[i, j]['weight'] = adj_matrix[i, j]

tsp = Tsp(graph)
print("TSP Graph".format(n_cities))
print("Adjacency Matrix:")
print(adj_matrix)

qp = tsp.to_quadratic_program()
print("TSP Quadratic Program:")
print(qp.prettyprint())

qaoa_mes = QAOA(
    sampler=Sampler(),
    optimizer=COBYLA(maxiter=150)
)

optimizer = MinimumEigenOptimizer(qaoa_mes)
result = optimizer.solve(qp)

x = result.x
tsp_solution = tsp.interpret(x)

print("QAOA Solution:")
print("Binary vector: {}".format(x))
print("TSP tour: {}".format(tsp_solution))
print("Tour length: {}".format(tsp.tsp_value(tsp_solution, adj_matrix)))

exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)

exact_solution = tsp.interpret(exact_result.x)
print("Exact classical solution (brute force):")
print("Binary vector: {}".format(exact_result.x))
print("TSP tour: {}".format(exact_solution))
print("Tour length: {}".format(tsp.tsp_value(exact_solution, adj_matrix)))


Graph created for a 3 city TSP problem.
Adjacency Matrix:
[[  0.      36.2048 138.2529]
 [ 39.0742   0.      21.8243]
 [ 55.3781   3.436    0.    ]]
--------------------------------------------------
The Quadratic Program formulated from the TSP:
Problem name: TSP

Minimize
  39.0742*x_0_0*x_1_1 + 39.0742*x_0_0*x_1_2 + 55.3781*x_0_0*x_2_1
  + 55.3781*x_0_0*x_2_2 + 39.0742*x_0_1*x_1_0 + 39.0742*x_0_1*x_1_2
  + 55.3781*x_0_1*x_2_0 + 55.3781*x_0_1*x_2_2 + 39.0742*x_0_2*x_1_0
  + 39.0742*x_0_2*x_1_1 + 55.3781*x_0_2*x_2_0 + 55.3781*x_0_2*x_2_1
  + 3.436*x_1_0*x_2_1 + 3.436*x_1_0*x_2_2 + 3.436*x_1_1*x_2_0
  + 3.436*x_1_1*x_2_2 + 3.436*x_1_2*x_2_0 + 3.436*x_1_2*x_2_1

Subject to
  Linear constraints (6)
    x_0_0 + x_0_1 + x_0_2 == 1  'c0'
    x_1_0 + x_1_1 + x_1_2 == 1  'c1'
    x_2_0 + x_2_1 + x_2_2 == 1  'c2'
    x_0_0 + x_1_0 + x_2_0 == 1  'c3'
    x_0_1 + x_1_1 + x_2_1 == 1  'c4'
    x_0_2 + x_1_2 + x_2_2 == 1  'c5'

  Binary variables (9)
    x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_

IBMInputValueError: 'The instruction QAOA on qubits (0, 1, 2, 3, 4, 5, 6, 7, 8) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'